# Init & Login

In [3]:
from math import ceil
from random import randrange
from time import sleep

import spotipy
import spotipy.util as util
from IPython.display import clear_output

## Client Info ##
CLIENT_ID     = ""
CLIENT_SECRET = ""
CLIENT_SCOPE  = "user-follow-modify playlist-modify-private playlist-modify-public"
USER_NAME     = "31ytgsr7wdmiaroy77msqpiupdsi"
REDIR_URI     = "https://github.com/jwatson-CO-edu/yt_shuffle_so_good"
AUTH_URL      = 'https://accounts.spotify.com/api/token'
BASE_URL      = 'https://api.spotify.com/v1/'
## API Info ##
_RESPONSE_LIMIT = 100

with open( "../keys/spot_ID.txt" , 'r' ) as f:
    CLIENT_ID = f.readlines()[0].strip()

with open( "../keys/spot_SECRET.txt" , 'r' ) as f:
    CLIENT_SECRET = f.readlines()[0].strip()

token = util.prompt_for_user_token(
    username      = USER_NAME,
    scope         = CLIENT_SCOPE,
    client_id     = CLIENT_ID,
    client_secret = CLIENT_SECRET,
    redirect_uri  = REDIR_URI
)

spot = spotipy.Spotify( auth = token )
clear_output( wait = False )

# Playlist Functions

In [8]:

def get_playlist_length( playlist_ID ):
    """ Get the number of total tracks in the playlist """
    response = spot.user_playlist_tracks(
        CLIENT_ID, 
        playlist_ID, 
        fields = 'items,uri,name,id,total', 
        limit  = _RESPONSE_LIMIT
    )
    return response['total']


def reorder_playlist_by_chunks( playlist_ID, chunkSize, Nmoves = None, verbose = False ):
    """ Shuffle the playlist a chunk at a time """
    Ntracks = get_playlist_length( playlist_ID )
    Nchunks = int( ceil( Ntracks / chunkSize ) )
    if Nmoves is None:
        Nmoves = int( Ntracks / chunkSize )
    for i in range( Nmoves ):
        bgnDex   = randrange(0,Nchunks) * chunkSize
        moveSize = min( chunkSize, Ntracks - bgnDex )
        sendDex  = randrange(0,Ntracks)
        try:
            response = spot.playlist_reorder_items(
                playlist_ID, 
                bgnDex, 
                sendDex, 
                range_length = chunkSize
            )
            if verbose:
                print( f"Iteration {i+1}, {bgnDex}-{bgnDex+moveSize} --> {sendDex}:\n\t{response}" )
        except spotipy.SpotifyException as ex:
            if verbose:
                print( f"ERROR, {bgnDex}-{bgnDex+moveSize} --> {sendDex}:\n\t{ex}" )


def complete_reorder_entire_playlist( playlist_ID, verbose = False, phasePause_s = 5 ):
    """ Completely shuffle the playlist in decreasing chunks """
    Ntracks = get_playlist_length( playlist_ID )
    ## Phase 1: Chunks of 100 ##
    if verbose:
        print("\nPhase 1: Chunks of 100\n")
    reorder_playlist_by_chunks( playlist_ID, 100, verbose = verbose )
    sleep( phasePause_s )
    ## Phase 2: Chunks of 10 ##
    if verbose:
        print("\nPhase 2: Chunks of 10\n")
    reorder_playlist_by_chunks( playlist_ID, 10, verbose = verbose )
    sleep( phasePause_s )
    ## Phase 3: Chunks of 5, 1/2 ##
    if verbose:
        print("\nPhase 3: Chunks of 5, 1/2\n")
    Nmoves = int( ceil( Ntracks / 10 ) )
    reorder_playlist_by_chunks( playlist_ID, 5, Nmoves, verbose = verbose )
    sleep( phasePause_s )
    ## Phase 3: 1 at a Time, 1/3 ##
    if verbose:
        print("\nPhase 3: 1 at a Time, 1/3\n")
    Nmoves = int( ceil( Ntracks / 3 ) )
    reorder_playlist_by_chunks( playlist_ID, 1, Nmoves, verbose = verbose )

# Shuffle Playlists!

In [9]:
playlist = {
    'study01' : "0a2qoe6S7lYeZ6nlhZdA0v",
    'study02' : "6gbtR2cBq5PvkghidCvvGk",
    'study03' : "3o3lN2qntdEV7UKTuuC77K",
    'study04' : "41sFSisljvBDMBXtpp5NIw",
    'study05' : "02iS5AFGp8YVuUUqcQf8ys",
    'study06' : "6KI7A4MWrSM7EyKRUjxIi1",
    'study07' : "3V055Md2JdrUT8tX0af7di",
    'study08' : "0tspdJlwSgiyf2O9PO6QaP",
    'study09' : "5mHRBFoQtYy2izeZ66pG95",
    'study10' : "3832xeKGEOAXFJqE4K8kIq",
    'study11' : "65MXR4dubPL9t0P4dgTWvn",
}

complete_reorder_entire_playlist( playlist['study05'], verbose = True, phasePause_s = 5 )


Phase 1: Chunks of 100

Iteration 1, 200-300 --> 17:
	{'snapshot_id': 'ODQzLGZlMDgyMTI2NTRjODFiN2NkYTMxZmQ2NThmNzU5ZDIwN2Q0OGZiNzQ='}
Iteration 2, 200-300 --> 247:
	{'snapshot_id': 'ODQ0LGUzY2FlZTUzMzgxMWM2MGFlMzY2MzI4NWM5MGFkNjQwZmViZDc3OTM='}
Iteration 3, 0-100 --> 3:
	{'snapshot_id': 'ODQ1LDA4ODYxYzc3YjM5NmE1OTU0YmRjN2RhMWZiMzMxN2NjOGNjNTY2YmQ='}

Phase 2: Chunks of 10

Iteration 1, 130-140 --> 132:
	{'snapshot_id': 'ODQ2LDhkMjRlNDdlZDMyNTdiNjk5YThhODIwODA1MGIzZTViNDA0NGEzYjY='}
Iteration 2, 280-290 --> 8:
	{'snapshot_id': 'ODQ3LDc5N2MzOGM4Mzg1ZGMwODhhMmE1MWE4Y2Q0N2FmMjJiNWVmNDg4ODM='}
Iteration 3, 40-50 --> 59:
	{'snapshot_id': 'ODQ4LGU4YWJlYzM1M2ViNGM4MTExYjAyOGQxYzhkM2RlOWYyMDg4Mjc4ZDM='}
Iteration 4, 250-260 --> 308:
	{'snapshot_id': 'ODQ5LGFmNzgzOGI0Nzk0OWVjYjk5M2UzNTcxNjg5NTE0NTRjNjY2NGM5NzI='}
Iteration 5, 30-40 --> 226:
	{'snapshot_id': 'ODUwLDYyNTM5ZGQ0NmI3MTFjZDQyZDU2ZTY2OTNlMTYxY2Q4ZWE2ZDk5ZTM='}
Iteration 6, 290-300 --> 260:
	{'snapshot_id': 'ODUxLDMwZTk3Njc0MGFhZmNjOTZ

HTTP Error for PUT to https://api.spotify.com/v1/playlists/02iS5AFGp8YVuUUqcQf8ys/tracks with Params: {} returned 400 due to Tracks selected to be reordered are out of bounds


Iteration 22, 200-205 --> 111:
	{'snapshot_id': 'OTAxLGViZGQxMDUyN2EzODdlZGE1MzA1NWU1N2I0OWEwNWI3NTg0ZDQ2YTU='}
ERROR, 340-341 --> 217:
	http status: 400, code:-1 - https://api.spotify.com/v1/playlists/02iS5AFGp8YVuUUqcQf8ys/tracks:
 Tracks selected to be reordered are out of bounds, reason: None
Iteration 24, 240-245 --> 139:
	{'snapshot_id': 'OTAyLDIyOTRiYTMwMjhiNWM5MmM3ZGEwMGJlZDgyODBiMDg5YjNmMzQyMjk='}
Iteration 25, 285-290 --> 305:
	{'snapshot_id': 'OTAzLDFjNmM3NjI2NTUwZWRhYWI3NTZjN2RiZWNiNmEzZGUzZThlNzhkMzE='}
Iteration 26, 130-135 --> 306:
	{'snapshot_id': 'OTA0LDUwMGE3MmUxOGM4YTM3N2IzZGVmNzRmNTkzMjZjMGU1NWFiZmQxYjU='}
Iteration 27, 50-55 --> 279:
	{'snapshot_id': 'OTA1LGMyNTNhNzg1MmE0MjQ4YTgyOGY2NWRjMWI2NTNlZWE3YTJkNWMwYTM='}
Iteration 28, 160-165 --> 93:
	{'snapshot_id': 'OTA2LGRlMTI4Zjc1MTIxZmVmMWQ0NTkzMjk1ZWNkMzVhY2NjMWU1OGVkYWY='}
Iteration 29, 335-340 --> 209:
	{'snapshot_id': 'OTA3LGNmN2RlZDMxNzJkNmJiOTUwNTE3NjgzZmNhNGRhNWZmOTkyNWM0Mzg='}
Iteration 30, 285-290 --> 59:
	{'